<a href="https://colab.research.google.com/github/Orneyfish/Orney/blob/master/BERT_SENT_EMB_RESEARCH_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install bert-serving-server bert-serving-client
! pip install tensorflow-gpu==1.14.0-rc0

     |████████████████████████████████| 71kB 2.5MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=96381d8e909be45b660bd860d87ca01536ad08162ce7cad9ae23e62268238ebe
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.14.0-rc0 (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 1.3.0, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.9.0, 1.10.0, 1.10.1, 1.11.0, 1.12.0, 1.12.2, 1.12.3, 1.13.1, 1.13.2, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4)
ERROR: No matching distribution found for tensorflow-gpu==1.14.0-rc0


In [2]:
# Install the library using pip
!pip install sentence-transformers

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 573kB 6.0MB/s 
     |████████████████████████████████| 3.7MB 8.9MB/s 
     |████████████████████████████████| 1.0MB 37.8MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=7110399da61065150871a951a59c3bb258203edf6bbdd21d98ccab7d3434caac
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=341bc05bb1107e9758fb729d399526689081467989ed80dc31aae0e06dba2d54
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sentence-transformers sacremoses


In [3]:
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:19<00:00, 20.3MB/s]


In [4]:
import pandas as pd

df = pd.read_csv("/content/metadata_mod.csv", delimiter='\t', header=None, names=['title', 'abstract'], encoding='utf-8')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 11,398



,title,abstract
689,Smallpox and Season: Reanalysis of Historical ...,NaN
2385,Interferon-β 1a and SARS Coronavirus Replicati...,NaN
5492,Identification and pathogenicity of a variant ...,NaN
3175,From superspreaders to disease hotspots: linki...,NaN
7159,Epidemiology and Clinical Presentations of Res...,NaN
2235,"Human Bocavirus Infection among Children, Jord...",NaN
3452,What we are watching—five top global infectiou...,NaN
1476,The many paths to frameshifting: kinetic model...,NaN
5517,Learning from Ebola Virus: How to Prevent Futu...,NaN
2934,Complete genome sequence of an astrovirus iden...,NaN


In [5]:
df.sample(5)[['title', 'abstract']]

,title,abstract
2808,Blow Flies Were One of the Possible Candidates...,NaN
4669,Graph-distance distribution of the Boltzmann e...,NaN
10949,Emerging viruses and current strategies for va...,NaN
8303,A Stakeholder Survey on Live Bird Market Closu...,NaN
7070,Global Capacity for Emerging Infectious Diseas...,NaN


In [0]:
# Get the lists of sentences and their labels.
sentences = df.title.values
labels = df.abstract.values

In [7]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [8]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis,"Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused to different sequences (‘bodies’) derived from the 3′ end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synthesis that resembles copy-choice RNA recombination. During this process, the nascent RNA strand is transferred from one site in the template to another, during either plus or minus strand synthesis, to yield subgenomic RNA molecules. Central to this process are transcription-regulating sequences (TRSs), which are present at both template sites and ensure the fidelity of strand transfer. Here we present results of a comprehensive co-variation mutagenesis study of equine arteritis virus TRSs, demonstrating that discontinuous RNA synthesis depends not only on base pairing between sense leader TRS and antisense body TRS, but also on the primary

In [9]:
# A corpus is a list with documents split by sentences.

# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector - note includes negative values [-3.38723838e-01  2.96061873e-01 -2.78483331e-01 -1.45945564e-01
  5.89729726e-01 -5.45882761e-01  3.15661192e-01 -5.41660190e-01
  4.05491620e-01  1.61344185e-02  3.68212253e-01 -1.36691749e-01
 -1.04050040e-01  7.50824749e-01 -7.48443484e-01  1.61571518e-01
 -6.38414562e-01 -3.42398554e-01  3.36153746e-01  9.46062729e-02
 -6.72909975e-01  2.77225047e-01 -2.88485587e-01  3.98393273e-01
  9.46157813e-01  8.09267342e-01  3.76242518e-01 -2.30384335e-01
 -2.96445131e-01  1.73310935e-01 -5.99181987e-02 -3.02048564e-01
  2.82107860e-01 -4.48786944e-01 -4.60835040e-01  4.48058039e-01
 -1.59947261e-01  1.44201964e-01  5.30580342e-01  2.78250396e-01
 -2.30125442e-01 -1.00641084e+00 -4.93438542e-03  1.01648547e-01
 -8.51575077e-01 -7.52559304e-01 -1.80457547e-01  1.41855133e+00
 -8.39680314e-01 -1.14893809e-01 -6.91409945e-01 -1.98581591e-01
  6.32133067e-01  1.00627564e-01  4.21162814e-01  1.

In [10]:
!pip install sklearn
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [13]:
query = 'research on Machine learning' #@param {type: 'string'}

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest sentences of the corpus for each query sentence based on cosine similarity
number_top_matches =  5#@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(),  "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: research on Machine learning

Top 5 most similar sentences in corpus:
Hierarchical Clustering Using the Arithmetic-Harmonic Cut: Complexity and Experiments,"Clustering, particularly hierarchical clustering, is an important method for understanding and analysing data across a wide variety of knowledge domains with notable utility in systems where the data can be classified in an evolutionary context. This paper introduces a new hierarchical clustering problem defined by a novel objective function we call the arithmetic-harmonic cut. We show that the problem of finding such a cut is [Image: see text]-hard and [Image: see text]-hard but is fixed-parameter tractable, which indicates that although the problem is unlikely to have a polynomial time algorithm (even for approximation), exact parameterized and local search based techniques may produce workable algorithms. To this end, we implement a memetic algorithm for the problem and demonstrate the effectiv